In [13]:
# ==================================
# Import data manipulation Packages
import pandas as pd
# from jupyter_datatables import init_datatables_mode
import numpy as np
import sweetviz as sv
import os

# ===============================
# IPython display
import ipywidgets as widgets
from IPython.display import display, IFrame


# ==================================
# utilities
from pathlib import Path
from collections import defaultdict

In [2]:
# ===========================
# - locate relevant data files from the data directory.
# - Build a data dictionary of dir and files
# Specify the directory to search in
base_directory = Path('./Data')

# Create a defaultdict to automatically handle lists as values
files_by_folder = defaultdict(list)

# Iterate over all files in the specified directory and subdirectories
for file in base_directory.rglob('*'):
    if file.is_file():  # Check if the item is a file
        # Use the parent directory as the key and append the file to the list
        files_by_folder[file.parent.name].append(file.name)

# Convert defaultdict to a regular dict if you don't need the automatic list handling anymore
files_by_folder = dict(files_by_folder)

# Print or use the dictionary as needed
print(f"This is the folder file dictionary:\n{files_by_folder}\n\n")

print("\tFiles List\n", 3*"----------")
for file in files_by_folder["Data"]:
    print(file)
    


This is the folder file dictionary:
{'Data': ['titanic_dataset.csv', 'PIH-02.txt', 'PIH-01.txt']}


	Files List
 ------------------------------
titanic_dataset.csv
PIH-02.txt
PIH-01.txt


In [3]:
# =================================================================
# - Create the full file path from the files_by_folders dictionary
# - Read one of the Data files into pandas Dataframe
# - Explore the Data
# - Print out the column names
# - Profile the Data in each column
# - Think about Feature Engineering

# =================================================
# construct a file path 
# get list of directories from the data dictionary
directories = list(files_by_folder.keys())

# extract the name of first directory
directory = directories[0]

# extract the data file you want from the 
# list of files in the directory
file = files_by_folder[directory][1]

# create the full file path name to be 
# passed to the pandas_dataframe_tool function
# to extract the file into a pandas dataframe
file_path = "./" + directory+"/"+ file


# ==============================
# Function to read the file into 
# a pandas dataframe
def pandas_dataframe_tool(file_path: str):
  
    # load either .csv or .txt data file
    file_extension = Path(file_path).suffix.lower()
    try: 
        # losd csv file
        if file_extension == '.csv':
            df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)
        
        # load txt file
        elif file_extension == '.txt':
            df = pd.read_csv(file_path, delimiter='\t')

        else:
            return "Error: Unsupported file format. Please use a .csv or .txt file."
        
        return df
    
    except FileNotFoundError:
        return f"Error: The file '{file_path}' was not found."
    except pd.errors.EmptyDataError:
        return "Error: The file is empty"
    except pd.errors.ParserError:
        return "Error: There is a problem parsing the file."
    except Exception as e:
        return f"Error: {str(e)}"

# pandas dataframe
df = pandas_dataframe_tool(file_path=file_path)

In [4]:
# Feature Engineering
def determine_shift(hour):
    if 0 <= hour < 8:
        return  "1st Shift"
    elif 8 <= hour < 16:
        return "2nd Shift"
    else:
        return "3rd Shift"
        
def split_date_time_column(df, column_name: str) -> pd.DataFrame:
    # Convert 'Date/Time' column to datetime format if not already
    df['Date_Time'] = pd.to_datetime(df[column_name], format='%m/%d/%Y %I:%M:%S %p')
        
    # Extract Month, Day, Year, and Time into new columns using .dt
    df['Month'] = df['Date_Time'].dt.month
    df['Day'] = df['Date_Time'].dt.day
    df['Year'] = df['Date_Time'].dt.year
    df['Time'] = df['Date_Time'].dt.time
    df['Hour'] = df['Date_Time'].dt.hour
    # determine shift
    df['Shift'] = df['Hour'].apply(determine_shift)
    # determine day of the week
    df['Day_of_Week'] =df['Date_Time'].dt.day_name()
    df = df.drop(columns=[column_name])
    return df

In [5]:
# Build an interactive tool to explor and modify 
# the pandas dataframe just created
df

,Date_Time,FracSec,Event Type,Event SubType,Category,Area,Node,Unit,Module,Module Description,Attribute,State,Event Level,Desc1,Desc2,Archived
0,11/12/2023 7:38:47 PM,5387,ALARM,NaN,PROCESS,UTILITIES,DVC-040002,VE-7410,PU-7413,VE-7410 Waste Sump Pump,FAIL_ALM,INACT/UNACK,11-WARNING,FAILED,Clear,0
1,11/12/2023 7:38:47 PM,6934,EVENT,NaN,SYSTEM,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_WASH,DISABLED,NaN,ANY,Any Alarm Value %P1,0
2,11/12/2023 7:38:49 PM,5344,EVENT,NaN,SYSTEM,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_RINSE,INACT/ACK,NaN,ANY,Any Alarm Value %P1,0
3,11/12/2023 7:38:49 PM,9354,EVENT,NaN,PROCESS,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_RINSE/BSTATUS,RUNNING,4-INFO,NaN,NaN,0
4,11/12/2023 7:39:09 PM,954,EVENT,NaN,SYSTEM,AREA_A,DVC-040025,NaN,NaN,NaN,REDU,NaN,NaN,00-22-E5-32-43-EA,Standby is now Unavailable,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392726,12/1/2023 4:59:46 AM,1307,ALARM,NaN,PROCESS,DSP2_CIP,DVC-040018,SK-046210,PI-04623-18,CIP Skid Supply Pressure,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 33.4993 Limit 1,0
392727,12/1/2023 4:59:46 AM,1797,ALARM,NaN,PROCESS,DSP2_CIP,DVC-040018,SK-046210,FIC-04623-17,CIP Skid Supply Flowrate,HI_HI_ALM,INACT/ACK,11-WARNING,HIHI,High High Alarm Value 185.321 Limit 350,0
392728,12/1/2023 4:59:46 AM,1807,ALARM,NaN,PROCESS,DSP2_CIP,DVC-040018,SK-046210,FIC-04623-17,CIP Skid Supply Flowrate,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 185.321 Limit 1,0
392729,12/1/2023 4:59:46 AM,3287,ALARM,NaN,PROCESS,DSP2_CIP,DVC-040018,SK-046210,VI-04621-03,VE-046210 Volume,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 1029.27 Limit 25,0


In [6]:
for column_name in df.columns.to_list():
    print(column_name)

Date_Time
FracSec
Event Type
Event SubType
Category
Area
Node
Unit
Module
Module Description
Attribute
State
Event Level
Desc1
Desc2
Archived


In [7]:
# ================================================
# analyze the dataframe and perform data cleaning
report = sv.analyze(df)
# report.show_notebook()
report.show_html("sweetviz_report.html")

                                             |          | [  0%]   00:00 -> (? left)

Report sweetviz_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [8]:
# =========================
# Drop unessary columns
df.drop(['Event SubType', 'Archived'], axis=1, inplace=True )

In [9]:
# =============================
# Feature Engineering 
df = split_date_time_column(df, column_name="Date_Time")

In [10]:
df

,FracSec,Event Type,Category,Area,Node,Unit,Module,Module Description,Attribute,State,Event Level,Desc1,Desc2,Month,Day,Year,Time,Hour,Shift,Day_of_Week
0,5387,ALARM,PROCESS,UTILITIES,DVC-040002,VE-7410,PU-7413,VE-7410 Waste Sump Pump,FAIL_ALM,INACT/UNACK,11-WARNING,FAILED,Clear,11,12,2023,19:38:47,19,3rd Shift,Sunday
1,6934,EVENT,SYSTEM,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_WASH,DISABLED,NaN,ANY,Any Alarm Value %P1,11,12,2023,19:38:47,19,3rd Shift,Sunday
2,5344,EVENT,SYSTEM,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_RINSE,INACT/ACK,NaN,ANY,Any Alarm Value %P1,11,12,2023,19:38:49,19,3rd Shift,Sunday
3,9354,EVENT,PROCESS,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_RINSE/BSTATUS,RUNNING,4-INFO,NaN,NaN,11,12,2023,19:38:49,19,3rd Shift,Sunday
4,954,EVENT,SYSTEM,AREA_A,DVC-040025,NaN,NaN,NaN,REDU,NaN,NaN,00-22-E5-32-43-EA,Standby is now Unavailable,11,12,2023,19:39:09,19,3rd Shift,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392726,1307,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,PI-04623-18,CIP Skid Supply Pressure,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 33.4993 Limit 1,12,1,2023,04:59:46,4,1st Shift,Friday
392727,1797,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,FIC-04623-17,CIP Skid Supply Flowrate,HI_HI_ALM,INACT/ACK,11-WARNING,HIHI,High High Alarm Value 185.321 Limit 350,12,1,2023,04:59:46,4,1st Shift,Friday
392728,1807,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,FIC-04623-17,CIP Skid Supply Flowrate,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 185.321 Limit 1,12,1,2023,04:59:46,4,1st Shift,Friday
392729,3287,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,VI-04621-03,VE-046210 Volume,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 1029.27 Limit 25,12,1,2023,04:59:46,4,1st Shift,Friday


In [11]:
# =================================================================
# analyze the dataframe after performing the data cleaning 
# inspect the results from data cleaning and feature engineering.
report_cleaned_df = sv.analyze(df)
# report.show_notebook()
report_cleaned_df.show_html("sweetviz_report.html")

                                             |          | [  0%]   00:00 -> (? left)

Report sweetviz_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [17]:
# =================================
# write the cleaned data to disk as
# a csv file.
write_df_dir = os.getcwd() +'/Data'
write_df_file_name = "Clean_Event_Log.csv"
file_path = os.path.join(write_df_dir, write_df_file_name)
df.to_csv(file_path, index=False)

/Users/jerrymegaro/Library/CloudStorage/OneDrive-Proconex/AI-DeltaV Event Logs/Data/Clean_Event_Log.csv


In [18]:
df

,FracSec,Event Type,Category,Area,Node,Unit,Module,Module Description,Attribute,State,Event Level,Desc1,Desc2,Month,Day,Year,Time,Hour,Shift,Day_of_Week
0,5387,ALARM,PROCESS,UTILITIES,DVC-040002,VE-7410,PU-7413,VE-7410 Waste Sump Pump,FAIL_ALM,INACT/UNACK,11-WARNING,FAILED,Clear,11,12,2023,19:38:47,19,3rd Shift,Sunday
1,6934,EVENT,SYSTEM,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_WASH,DISABLED,NaN,ANY,Any Alarm Value %P1,11,12,2023,19:38:47,19,3rd Shift,Sunday
2,5344,EVENT,SYSTEM,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_RINSE,INACT/ACK,NaN,ANY,Any Alarm Value %P1,11,12,2023,19:38:49,19,3rd Shift,Sunday
3,9354,EVENT,PROCESS,MEDIA_PREP,DVC-040010,NaN,VE-2110,Media Prep Component Tank 560L,MP_CIP_RINSE/BSTATUS,RUNNING,4-INFO,NaN,NaN,11,12,2023,19:38:49,19,3rd Shift,Sunday
4,954,EVENT,SYSTEM,AREA_A,DVC-040025,NaN,NaN,NaN,REDU,NaN,NaN,00-22-E5-32-43-EA,Standby is now Unavailable,11,12,2023,19:39:09,19,3rd Shift,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392726,1307,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,PI-04623-18,CIP Skid Supply Pressure,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 33.4993 Limit 1,12,1,2023,04:59:46,4,1st Shift,Friday
392727,1797,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,FIC-04623-17,CIP Skid Supply Flowrate,HI_HI_ALM,INACT/ACK,11-WARNING,HIHI,High High Alarm Value 185.321 Limit 350,12,1,2023,04:59:46,4,1st Shift,Friday
392728,1807,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,FIC-04623-17,CIP Skid Supply Flowrate,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 185.321 Limit 1,12,1,2023,04:59:46,4,1st Shift,Friday
392729,3287,ALARM,PROCESS,DSP2_CIP,DVC-040018,SK-046210,VI-04621-03,VE-046210 Volume,LO_LO_ALM,INACT/ACK,11-WARNING,LOLO,Low Low Alarm Value 1029.27 Limit 25,12,1,2023,04:59:46,4,1st Shift,Friday
